In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


In [2]:
df = pd.read_csv('data/dataset.csv').drop('date', axis=1)

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, labels, sequence_length):
        self.data = data
        self.labels = labels
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length + 1

    def __getitem__(self, idx):
        end_idx = idx + self.sequence_length
        return {
            'data': torch.FloatTensor(self.data[idx:end_idx]),
            'label': torch.FloatTensor(self.labels[end_idx - 1])
        }

# Supongamos que tienes 'data' con forma (n_samples, n_timesteps, n_features)
# y 'labels' con las etiquetas correspondientes.

# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Creación de conjuntos de datos de series temporales
train_dataset = TimeSeriesDataset(X_train, y_train, sequence_length=n_timesteps)
test_dataset = TimeSeriesDataset(X_test, y_test, sequence_length=n_timesteps)

# Creación de DataLoader para facilitar el entrenamiento
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Ahora puedes utilizar train_dataloader y test_dataloader para entrenar tu modelo.


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # La entrada x debe tener dimensiones (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(x)
        
        # Tomamos el último timestep de la salida de la LSTM
        last_timestep = lstm_out[:, -1, :]
        
        # Pasamos la salida del último timestep a través de una capa lineal
        output = self.fc(last_timestep)
        
        return output

# Definimos las dimensiones de entrada y salida de la red
input_size = 17  # Número de variables en cada timestep
hidden_size = 64  # Tamaño del estado oculto de la LSTM
num_layers = 2  # Número de capas en la LSTM
output_size = 1  # Tamaño de la salida (pronóstico para el próximo timestep)

# Creamos una instancia del modelo
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Definimos la pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


